In [12]:
import networkx as nx
import os
import pickle
from tqdm import tqdm
import numpy as np
import random
from collections import defaultdict
import copy

In [13]:
# IC MODEL
PROPOGATION = 0.1
total_runs = 30
ITERATIONS = 100
SEED_SIZE = 25
POPULATION_SIZE = 500

In [14]:
# for 
graphnames = ['spa_500_{}'.format(graphidx) for graphidx in range(50)]
GRAPHS = []
for path in os.listdir("fair_influmax_code_release/networks"):
    g = pickle.load(open(f"fair_influmax_code_release/networks/{path}", 'rb'))

    if 'spa' not in graphnames[0]:
        to_remove = []
        for v in g.nodes():
            if 'race' not in g.node[v]:
                to_remove.append(v)
        g.remove_nodes_from(to_remove)

    for u,v in g.edges():
        g[u][v]['p'] = PROPOGATION

    g = nx.convert_node_labels_to_integers(g, label_attribute='pid')
    GRAPHS += [g]

In [15]:

class Wolf:
    def __str__(self):
        return self.objective_values
    def __float__(self):
        return self.objective_values
    def __init__(self,g,metric, kind, name=None):
        self.g = g
        self.g_len = len(self.g.nodes)
        self.start = set(np.random.choice(range(self.g_len), SEED_SIZE,replace=False))
        self.metric = metric
        self.v = set(range(self.g_len)) - self.start
        
        self.c = self.get_property(range(self.g_len))
        self.start_c = self.get_property(self.start)
        self.kind = kind
        
        self.name = name
        self.domination_count = 0
        self.dominated_wolves = set()
        self.objective_values = []
    #maximize
    def influence_metric(self):
        self.influence = len(self.next)
        return self.influence
    #maximize
    def maximin_fairness_metric(self):
        self.maximin_fairness = np.inf
        for kind, next_list in self.next_c.items():
            start_list = self.c[kind]    
            if self.maximin_fairness > len(next_list)/len(start_list):
                self.maximin_fairness = len(next_list)/len(start_list)
        return self.maximin_fairness
    #maximize
    def group_rationality_metric(self):
        self.group_rationality = self.influence
        for kind, next_list in self.next_c.items():
                                
            subgraph = self.g.subgraph(self.c[kind])
            
            activated_nodes = self.start_c[kind]
            newly_activated = self.start_c[kind]
            idx = 1 
            # self.group_activation_dict = defaultdict(list)
            while newly_activated:
                next_round_activation = set()
                for node in newly_activated:
                    neighbors = set(subgraph.neighbors(node)) - activated_nodes
                    for neighbor in neighbors:
                        if random.random() < PROPOGATION:
                            next_round_activation.add(neighbor)
                activated_nodes.update(next_round_activation)
                newly_activated = next_round_activation
                
            if len(next_list) <= len(activated_nodes):
                self.group_rationality = 0
        return self.group_rationality
    
    #maximize
    def group_activation_speed_metric(self):
        self.group_activation_speed = np.inf
        if self.component[0]==0:
            self.group_activation_speed = 0
        else:
            for key, value in self.c.items():
                speed_key = np.dot(np.array(self.group_activation_dict[key]),self.component)/(self.component[0]*(len(value)-len(self.start_c[key])))                
                if speed_key < self.group_activation_speed:
                    self.group_activation_speed = speed_key
            
            if self.group_activation_speed == np.inf:
                self.group_activation_speed = 0
        return self.group_activation_speed
    def ic_model(self, g=None, start=None):
        if g==None:
            g = self.g
        if start == None:
            start = self.start.copy()
        activated_nodes = start
        newly_activated = start
        idx = 1 
        self.group_activation_dict = defaultdict(list)
        while newly_activated:
            next_round_activation = set()
            for node in newly_activated:
                neighbors = set(g.neighbors(node)) - activated_nodes
                for neighbor in neighbors:
                    if random.random() < PROPOGATION:
                        next_round_activation.add(neighbor)
            activated_nodes.update(next_round_activation)
            newly_activated = next_round_activation
            
            nij = self.get_property(newly_activated)
            
            for key,value in self.c.items():
                self.group_activation_dict[key] += [len(nij[key])] 
            idx += 1 
            
        component = list(range(0,idx-1))
        component.reverse()
        self.component = np.array(component)
        
        self.next = activated_nodes
        self.next_c = self.get_property(self.next)
        
        
        self.objective_values = [self.influence_metric()]
        if "maximin" in self.kind:
            self.objective_values+=[self.maximin_fairness_metric()]
        if "speed" in self.kind:
            self.objective_values+=[self.group_activation_speed_metric()]
        if "rationality" in self.kind:
            self.objective_values+=[self.group_rationality_metric()]
            
        
        self.objective_values = np.round(self.objective_values,3)

        
    def get_next_start(self,A):
        global DEBUG
        if A<1:
            e_n = self.leader - self.start
            e_o = self.start - self.leader
            d = len(e_o)
            step = int(np.ceil((1-A)*d))
            step = min(step, len(e_o), len(e_n))
            self.start = self.start - set(random.sample(list(e_o), step))
            self.start = self.start.union(set(random.sample(list(e_n), step)))
        else:
            if len(self.v)<self.g_len*0.8:
                self.v = set(range(self.g_len)) - self.start
            # if DEBUG==self:
            #     print("V:", len(self.v),"X:", len(self.start),"Leader:", len(self.leader))
            e_n = self.v - self.start.union(self.leader)
            e_o = self.start.intersection(self.leader) 
            d = len(self.start - self.leader)
            
            step = int(np.ceil((A-1)*d))
            # if DEBUG==self:
            #     print("e_n:", len(e_n),"e_o:", len(e_o),"step:", step)
            step = min(step, len(e_o), len(e_n))
            self.start = self.start - set(random.sample(list(e_o), step))
            # if DEBUG==self:
            #     print("Removing start: ", len(self.start))
            v_step = set(random.sample(list(e_n), step))
            self.start = self.start.union(v_step)
            # if DEBUG==self:
            #     print("Adding start: ", len(self.start))
            self.v = self.v - v_step 
            # if DEBUG==self:
            #     print("New V: ", len(self.v))
            #     print("\n\n")
                    
    def get_property(self,node_set):
        property_dict = defaultdict(list)
        for node in node_set:
            property_value = self.g.nodes[node][self.metric]
            if property_value not in property_dict:
                property_dict[property_value] = []
            property_dict[property_value].append(node)

        for key, value in property_dict.items():
            property_dict[key] = set(value)
        return property_dict
    
    
    
    def get_leader(self, wolves):
        min_difference = float('inf')
        self.leader = set()
        for X in wolves:
            difference = len(self.start - X.start)
            if difference < min_difference:
                min_difference = difference
                self.leader = X.start.copy()
        self.leader = set(random.sample(list(self.leader), int((len(self.leader)*0.6))))

def find_pareto_fronts(wolves):
    objective_values = np.array([wolf.objective_values for wolf in wolves])
    domination_counts = np.sum(np.all(objective_values >= objective_values[:, np.newaxis], axis=2), axis=1)
    pareto_front = [copy.deepcopy(wolves[i]) for i, count in enumerate(domination_counts) if count == 1]
    return pareto_front

In [16]:
import time

def simulate(kind):
    results =[]

    for g in tqdm(GRAPHS):
        wolves = [Wolf(g, "age", {"maximin"}) for i in range(POPULATION_SIZE)]
        archive = []
        DEBUG = wolves[0]
        for i in range(ITERATIONS):
            a = 2 - i * (2/ITERATIONS)
            A = 2* a * np.random.rand(len(g.nodes())) - a
            C = 2* np.random.rand(len(g.nodes()))

            magnitude_A = np.linalg.norm(A)
            for wolf in wolves:
                
                wolf.ic_model()
                if len(wolf.start)<20:
                    wolf = Wolf(g, "age")
            
            archive = find_pareto_fronts(wolves+archive)
            
            if len(archive)>3:
                leaders = random.sample(archive, 3)
            else:
                leaders = archive
            # print(len(archive), len(leaders[0].start), leaders[0].objective_values)    
            
            now = time.time()
            for wolf in wolves:
                if wolf not in leaders:
                    wolf.get_leader(leaders)
                    wolf.get_next_start(magnitude_A)
            # print("-"*100)
            
        for arch in archive:
            results += [[arch.influence_metric(), arch.maximin_fairness_metric(), arch.group_rationality_metric(), arch.group_activation_speed_metric()]]
        results = np.array(results)
    return np.average(results, axis=0) + [sum(results[:,2]==0)/len(results)]


In [17]:
yaya = np.array([simulate(i) for i in range(10)])
yaya

 17%|█▋        | 4/24 [00:58<04:54, 14.73s/it]

In [ ]:
import concurrent.futures
    
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     results = executor.map(simulate, range(30))
